# Μια ολοκληρωμένη διαδικασία ταξινόμησης

Το scikit-learn έχει καθιερώσει την ορολογία που περιγράφει με κομψό τρόπο μια ολοκληρωμένη διαδικασία Μachine Learning, στην περίπτωσή που εξετάζουμε μια διαδικασία ταξινόμησης.

## Pipeline, Εκτιμητές και Μετασχηματιστές

Μια διαδικασία ML ή workflow ή pipeline αποτελείται από μια σεiρα μετασχηματιστών πάνω στα χαρακτηριστικά των δεδομένων που τελειώνει σε έναν εκτιμητή.

![](https://sv3nd.github.io/images/scikit-learn-pipeline-post-processing_sk_pipeline.png)

Οι μετασχηματιστές χρησιμοποιούνται για να κανουν την προεπεξεργασία (μέσω μετασχηματισμού) των δεδομένων. Είδαμε στο προηγούμενο notebook πέντε μετασχηματιστές: την επιλογή χαρακτηριστικών VarianceThreshold, δύο μετασχηματιστές κανονικοποίησης (τον scaler και τον min_max_scaler), τον εξισορροπητή με τυχαία υπερδειγματοληψία RandomOverSampler και την εξαγωγή χαρακτηριστικών PCA. Αν ανατρέξετε στο προηγούμενο notebook θα δείτε ότι κάνουν fit και transform στο train set και transform στο test set.

Οι μετασχηματιστές γενικα έχουν και αυτοί υπερ-παραμέτρους που επηρρεάζουν τη λειτουργία τους: ο VarianceThreshold είχε το κατώτερο κατώφλι διακύμανσης ο PCA τον αριθμό των κύριων συνιστωσών, ενώ ακόμα και οι scaler, min_max_scaler και RandomOverSampler έχουν αλλά δεν τις εξετάσαμε. Όπως έχουμε πει η επιλογή των υπερ-παραμέτρων (όπως το k του kNN) γίνεται μόνο εμπειρικά μέσω διασταυρούμενης επικύρωσης (cross-validation). Οι μετασχηματιστές και οι υπερπαράμετροι τους επιδρούν λοιπόν στη μορφή των δεδομένων.

Στο τέλος του pipeline VarianceThreshold - scaler - RandomOverSampler - PCA βάλαμε τον εκτιμητή - ταξινομητή MultiLayerPerceptron. O MLP έχει και αυτός υπερ-παραμέτρους και μάλιστα έναν πολύ μεγάλο αριθμό (18 για την ακρίβεια): πλήθος και επίπεδα κρυμμένων νευρώνων, συνάρτηση ενεργοποίησης, βελτιστοποίησης κλπ. Στο προηγούμενο notebook χρησιμοποιήσαμε ένα MLP με σταθερές υπερπαραμέτρους, ωστόσο σε μια ολοκληρωμένη διαδικασία ML οι υπερ-παράμετροι (ή κάποιες από τις υπερπαραμέτρους) του ταξινομητή πρέπει και αυτές να βελτιστοποιηθούν με διαδικασία cross-validation. Οι ταξινομητές και οι υπερπαράμετροι τους δεν επιδρούν στη μορφή των δεδομένων όπως οι μετασχηματιστές αλλά έχουν προφανώς επίδραση στην απόδοση του μοντέλου.

## Ορισμός (επιλογή) ενός εκπαιδευμένου μοντέλου εκτιμητή (ταξινομητή)

Μια ολοκληρωμένη διαδικασία pipeline λόγω του ότι τελειώνει σε έναν εκτιμητή μπορεί να θεωρηθεί και συνολικά ως ένας εκτιμητής, με κανένα, με λίγους ή με περισσότερους μετασχηματιστές πριν από τον εκτιμητή. Στο προηγούμενο παράδειγμα είδαμε στην αρχή ένα μοντέλο με μόνο τον εκτιμητή (εφαρμογή του MLP απευθείας στο dataset) και ένα τελικό μοντέλο με τέσσερεις μετασχηματιστές πριν τον εκτιμητή. Ένα εκπαιδευμένο μοντέλο εκτιμητή (ταξινομητή) αποτελείται
- α) απο την αρχιτεκτονική του, δηλαδή τον συνδυασμό μετασχηματιστών και την επιλογή του τελικού εκτιμητή (το pipeline), και 
- β) από τις (βέλτιστες) τιμές των υπερ-παραμέτρων όλων των προηγουμένων που προκύπτουν από το cross-validation. 

<img src="https://github.com/rasbt/pattern_classification/raw/master/Images/supervised_learning_flowchart.png" width="65%">

Το τελικό βελτιστοποιημένο μοντέλο αποτιμάται στα δεδομένα test και χρησιμοποιείται για να κάνει προβλέψεις σε νέα δεδομένα.

ΠΡΟΣΟΧΗ: εκτός από τη διαχείριση τιμών που απουσιάζουν με Imputer, όλοι οι υπόλοιποι μετασχηματιστές βρίσκονται εντός του σχήματος crossvalidation. 

Η μετατροπή κατηγορικών μεταβλητών γίνεται μετά τη διαχείριση τιμών που απουσιάζουν και πριν το crossvalidation (εκτός δλδ).

## Βελτιστοποίηση ύπερ-παραμέτρων (Hyperparameter optimization)

Είδαμε ότι τόσο οι μετασχηματιστές όσο και οι εκτιμητές έχουν υπερ-παραμέτρους που πρέπει να βελτιστοποιηθούν με cross-validation. Εφόσον τόσο οι μετασχηματιστές όσο και ο εκτιμητής αποτελούν μέρος ενός ενιαίου pipeline, για να βρούμε τις βέλτιστες τιμές όλων των υπερ-παραμέτρων μέσω cross-validation θα πρέπει
- α) για κάθε fold του cross-validation, να υπολογίσουμε την απόδοση όλων των πιθανών συνδυασμών υπερ-παραμέτρων μετασχηματιστών και εκτιμητή και 
- β) να επιλέξουμε το συνδυασμό υπερ-παραμέτρων που έχει τον καλύτερο μέσο όρο με βάση κάποια μετρική σε όλα τα folds.


### Αναζήτηση πλέγματος (grid search)

Η απόδοση όλων των πιθανών συνδυασμών υπερ-παραμέτρων μετασχηματιστών και εκτιμητή γίνεται με αναζήτηση πλέγματος (grid search). Ορίζουμε για κάθε παράμετρο ένα πεδίο ορισμού, συνήθως με ελάχιστο, μέγιστο και κάποιο βήμα και φτιάχνουμε ένα πλέγμα με όλους τους πιθανούς συνδυασμούς τιμών των παραμέτρων. Για παράδειγμα:

για κύριες συνιστώσες PCA (transformer) από 5 μέχρι 15 με βήμα 5 και για έναν kNN (estimator) με k από 1 μέχρι 5 με βήμα 2 παίρνουμε το ακόλουθο grid:

|     |       |        |        |
|-----|-------|--------|--------|
|     | PC=5  | PC=10  | PC=15  |
| k=1 | (1,5) | (1,10) | (1,15) |
| k=3 | (3,5) | (3,10) | (3,15) |
| k=5 | (5,5) | (5,10) | (5,15) |

Για κάθε τιμή υπερπαραμέτρων του grid θα πρέπει να υπολογιστεί ο μέσος όρος του εκτιμητή σε όλα τα folds του cross-validation με βάση το metric (πχ F1) και να επιλεχθεί ο καλύτερος συνδυασμός παραμέτρων. Η συγκεκριμένη στρατηγική αναζήτησης των βέλτιστων υπερπαραμέτρων είναι η εξαντλητική αναζήτηση πλέγματος (exhaustive grid search) και είναι προφανώς πολύ ακριβή υπολογιστικά. Υπάρχουν διάφορες τεχνικές για να περιορίζεται η πολυπλοκότητα του grid search, αλλά δεν το αποφεύγουμε γενικά, γιατί οι υπερπαράμετροι είναι ορίσματα των εκτιμητών και δεν μαθαίνονται από την fit.

Συνοψίζοντας, η βελτιστοποίηση των υπερπαραμέτρων απαιτεί 

- έναν εκτιμητή (έναν ταξινομητή)
- τον πεδίο ορισμού των υπερπαραμέτρων
- ένα τρόπο αναζήτησης των πιθανών συνδυασμών τιμών τους πχ grid search
- ένα σχήμα cross-validation πχ 5-fold
- μια μετρική απόδοσης (ή score) πχ F1-macro

Το scikit-learn μας απλοποιεί σε πολύ μεγάλο βαθμό την κατασκευή pipelines και τη βελτιστοποίηση των υπερπαραμέτρων. Θα το δούμε με ένα παράδειγμα.

Θα βελτιστοποιήσουμε με cross-validation και grid search ένα pipeline με προ-επεξεργασία των δεδομένων από transformers και estimator τον kNN.

# MNIST handwritten digits dataset
Ενημερώνουμε τις βιβλιοθήκες μας. Επίσης θα αγνοήσουμε κάποια warnings

In [0]:
!pip install -U scikit-learn
!pip install -U numpy
!pip install -U pandas
import warnings 
warnings.filterwarnings('ignore')


Ένας ακόμη τρόπος να εισάγουμε datasets είναι από το [mldata.org](http://mldata.org/), ένα ανοικτό αποθετήριο datasets. Θα εισάγουμε το πολύ γνωστό και απο το Deep Learning dataset [MNIST](http://yann.lecun.com/exdb/mnist/). 

In [0]:
# import numpy as np
# Αυτή τη στιγμή το mldata.org είναι πεσμένο
# from sklearn.datasets import fetch_mldata
# mnist = fetch_mldata('MNIST original')

Θα το εισάγουμε από έναν άλλο server όπου υπάρχει ένα αντίγραφο. Το dataset είναι επίσης διαθέσιμο στο [Kaggle](https://www.kaggle.com/gustavoatt/mnist-original).

In [0]:
import numpy as np
import pandas as pd

mnist_train = pd.read_csv("https://www.python-course.eu/data/mnist/mnist_train.csv", header=None).values
mnist_test = pd.read_csv("https://www.python-course.eu/data/mnist/mnist_test.csv", header=None).values
mnist = np.concatenate((mnist_train, mnist_test), axis=0) # ενώνουμε train και test
# εναλλακτικά αν έχει μεγάλο φόρτο το python-course
# mnist = pd.read_csv("https://dodo.islab.ntua.gr/mnist.csv", header=None).values

In [0]:
features = mnist[:, 1:]
targets = mnist[:, :1] # τα labels είναι στην πρώτη κολώνα

To MNIST περιλαμβάνει 70000 δείγματα χειρόγραφων ψηφίων μεγέθους 28x28 pixels, με ετικέτες από το 0 ως το 9. Τα 28x28 pixels κάθε δείγματος αντιστοιχούν σε 768 χαρακτηριστικά με τιμές του γκρι από 0 (μάυρο) εώς 256 (λευκό)

In [0]:
print(features.shape)
print(targets.shape)
print(np.unique(targets)) #τυπώνουμε τις μοναδικές ετικέτες των labels

Θα κάνουμε οπτικοποίηση ενός δείγματος:

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
pixels = features[50000]
pixels = pixels.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

Επειδή το dataset είναι μεγάλο (ειδικά για τον kNN) και για το παράδειγμά μας θέλουμε να δουλέψουμε με λιγότερα δείγματα. Το ανακατεύουμε και παίρνουμε ένα μικρό αριθμό samples

In [0]:
from sklearn.utils import shuffle
sdata, starget = shuffle(features, targets, random_state=341976)
samples = 1000
data = sdata[0:samples-1,:]
target = starget[0:samples-1]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=20176)

In [0]:
from sklearn import neighbors
from sklearn.metrics import classification_report
clf = neighbors.KNeighborsClassifier()
clf.fit(X_train,y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

# Pipelines

Για την κατασκευή του μοντέλου θα βασιστούμε στην κλάση Pipeline. Επειδή οι κλάσεις εξισορρόπησης του imblearn όπως η [RandomOverSampler](http://contrib.scikit-learn.org/imbalanced-learn/stable/generated/imblearn.over_sampling.RandomOverSampler.html) τυπικά δεν έχουν μέθοδο transform (έχουν fit_sample) η built-in Pipeline του scikit (from sklearn.pipeline import Pipeline) δεν τις δέχεται ως transformers. Θα φέρουμε την Pipeline από το imblearn (που έχει transform για τους samplers).

In [0]:
!pip install --upgrade imbalanced-learn

In [0]:
#from imblearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline

# φέρνουμε τις γνωστές μας κλάσεις για preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler # φέρνουμε τον StandarScaler ως transformer που έχει .transform kai ΄όχι ως scale()
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA

# αρχικοποιούμε τους εκτιμητές (μετασχηματιστές και ταξινομητή) χωρείς παραμέτρους
selector = VarianceThreshold()
scaler = StandardScaler()
ros = RandomOverSampler()
pca = PCA()
clf = neighbors.KNeighborsClassifier(n_jobs=-1) # η παράμετρος n_jobs = 1 χρησιμοποιεί όλους τους πυρήνες του υπολογιστή
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('kNN', clf)])

Το pipeline συμπεριφέρεται ως ένας ενιαίος estimator. Μπορούμε να εφαρμόσουμε fit και predict.

In [0]:
pipe.fit(X_train,y_train)
preds = pipe.predict(X_test)
print(classification_report(y_test, preds))

# GridsearchCV

Στη συνέχεια θα χρησιμποιήσουμε την [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) για να βελτιστοποιήσουμε τις υπερπαραμέτρους μας. Η GridSearchCV κάνει μαζί cross-validation και grid search. Αρχικά μελετάμε το variance των μετβλητών για τη variance threshold:

In [0]:
train_variance = X_train.var(axis=0)
print(train_variance)
print(np.max(train_variance))

 Την εισάγουμε και θέτουμε τις τιμές ορισμού των υπερπαραμέτρων:

In [0]:
from sklearn.model_selection import GridSearchCV

vthreshold = [0, 4000, 8000, 12000] #προσαρμόζουμε τις τιμές μας στο variance που παρατηρήσαμε
n_components = [10, 20, 30, 40, 50, 60]
k = [1, 6, 11, 21, 31, 41] # η υπερπαράμετρος του ταξινομητή

Επειδή ο χώρος αναζήτησης των βέλτιστων υπερπαραμέτρων αρχίζει να μεγαλώνει, ξαναορίζουμε την pipeline με την παράμετρο 'memory': για κάθε fold του crossvalidation και για καθε συνδυασμό υπερπαραμέτρων μετασχηματιστών, τα δεδομένα χρειάζεται να μετασχηματιστούν μία φορά και όχι για κάθε νέα τιμή υπερπαραμέτρων του εκτιμητή. 

Είναι πιθανό στο fit να σας εμφανιστούν κάποια warnings με τη χρήση του memory. Ξαναστρέξτε το block του κώδικα.

In [0]:
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('kNN', clf)], memory = 'tmp')

Μπορούμε να θέτουμε τιμές στις υπερπαραμέτρους των pipelines χρησιμοποιώντας τα ονόματα των estimators, "\_\_", το όνομα της υπερπαραμέτρου, "=" και τις τιμές που της δίνουμε στο grid search. Επίσης μπορούμε να θέσουμε τη μετρική της απόδοσης με την παράμετρο "scoring". Με την παράμετρο "cv" ορίζουμε τον αριθμό των folds. Για βελτιστοποίηση, μπορούμε να θέσουμε την παράμετρο n_jobs=-1 ώστε να χρησιμοποιούνται όλοι οι πυρήνες του υπολογιστή (το default είναι 1).

In [0]:
 estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold,pca__n_components=n_components, kNN__n_neighbors=k), cv=5, scoring='f1_macro', n_jobs=-1)

Το GridSearchCV είναι επίσης ένας estimator με fit και predict. Ανάλογα το search space η εκτέλεση του  GridSearchCV μπορεί να πάρει αρκετό χρόνο

In [0]:
import time
start_time = time.time()
estimator.fit(X_train, y_train)
preds = estimator.predict(X_test)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test, preds))

Tυπώνουμε τον καλύτερο estimator και τον καλύτερο συνδυασμό υπερπαραμέτρων:

In [0]:
print(estimator.best_estimator_)
print(estimator.best_params_)

Η στοχαστικότητα στη διαδικασία της ταξινόμησης οφείλεται στα διαφορετικό διαχωρισμό σε folds σε κάθε run αλλά στον RandomOverSampler που επιλέγει τυχαία δείγματα κατά τη δειγματοληψία.

## Επιλογή αρχιτεκτονικής μοντέλου pipeline

Προσοχή, η βέλτιστη αρχιτεκτονική δεν είναι δεδομένη αλλά εξαρτάται από το dataset. Δοκιμαστε στο ίδιο grid, χωρίς scaler και sampler

In [0]:
pipe = Pipeline(steps=[('selector', selector),('pca', pca), ('kNN', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components, kNN__n_neighbors=k), cv=3, scoring='f1_macro', n_jobs=-1)
# ο estimator με βελτιστοποιημένες υπερπαραμέτρους είναι έτοιμος να κάνει prediction.
# Ωστόσο για να μην πάνε χαμένα δεδομένα (ο΄ύτε ένα fold), τον κάνουμε fit σε όλα τα δεδομένα train.
estimator.fit(X_train, y_train)
preds = estimator.predict(X_test)
print(classification_report(y_test, preds))
print(estimator.best_estimator_)
print(estimator.best_params_)

In [0]:
estimator.get_params()

Και φυσικά θα μπορούσαμε να έχουμε διαφορετικά αποτελέσματα εφαρμόζοντας min max scaler αντι standard scaler, undersampling αντί oversampling κοκ. Προφανώς η πιο σημαντική απόφαση στην αρχιτεκτονική του ταξινομητή είναι η επιλογή του τελικού estimator,  αν πχ βάλουμε MLP ή SVM αντί kNN, και βέβαια η βελτιστοποίηση των υπερπαραμέτρων τους.

## Progressive grid search

Στο πεδίο ορισμού των παραμέτρων, ξεκινάμε με μεγάλα διαστήματα και σχετικά λίγα βήματα. Αν διαπιστώσουμε ότι υπαρχει μια περιοχή τιμών κάποιας παραμέτρου που δίνει καλη απόδοση μπορούμε να μικρύνουμε το διάστημα του grid search γύρω της και να βάλουμε περισσότερα βήματα.

In [0]:
vthreshold = [0]
n_components = [39, 40, 41]
k = [1, 3]
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components, kNN__n_neighbors=k), scoring='f1_macro', n_jobs=-1)
estimator.fit(X_train, y_train)
preds = estimator.predict(X_test)
print(classification_report(y_test, preds))
print(estimator.best_estimator_)
print(estimator.best_params_)

Το περισσότερο fine grained grid search, αν δώσει καλύτερες τιμές θα έχει βελτιστοποιήσει τον εκτιμητή, αν όχι, τουλάχιστον θα επιβεβαιώσει ότι είμαστε σε ένα καλό τοπικό μέγιστο της συνάρτησης αξιολόγησης. Θα βρείτε και άλλες συμβουλές για το grid search στο [FAQ](https://docs.google.com/document/d/1jL4gRag_LHbVCYIt5XVJ53iJPb6RZWi02rT5mPXiqEU/edit?usp=sharing) της πρώτης άσκησης.

---

